In [15]:
import numpy as np
import tensorflow as tf
import tensorflow.keras.backend as K

from mnist import MNIST
from sklearn.model_selection import train_test_split

from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.layers import Conv2D, MaxPool2D, GlobalAveragePooling2D, \
                                    Dense, Flatten, BatchNormalization, ReLU, Reshape, Lambda
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.python.compiler.tensorrt import trt_convert as trt

In [2]:
K.epsilon = 1e-5

In [3]:
# open data
mndata = MNIST('./python-mnist/data/')
images, labels = mndata.load_training()

# convert to numpy
images = np.array(images)
labels = np.array(labels)

# transform
images = images.reshape(-1,28,28)#.astype(float)/255
b = np.zeros((labels.size, labels.max()+1))
b[np.arange(labels.size),labels] = 1
labels = b

# check
images.shape, labels.shape

((60000, 28, 28), (60000, 10))

In [4]:
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.1, random_state=42)

# Model

In [21]:
model = Sequential()

model.add(Reshape((28,28,1), input_shape=(28,28)))
model.add(Lambda(lambda x: K.cast(x, "float32")/255))
model.add(Conv2D(8,kernel_size=(2,2)))
model.add(ReLU())
model.add(MaxPool2D(pool_size=(3,3)))
model.add(Conv2D(8,kernel_size=(2,2)))
model.add(ReLU())
model.add(MaxPool2D(pool_size=(3,3)))
model.add(Flatten())
model.add(Dense(10, activation='sigmoid'))

opt = Adam(learning_rate=0.00005)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

In [22]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape (Reshape)            (None, 28, 28, 1)         0         
_________________________________________________________________
lambda (Lambda)              (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d (Conv2D)              (None, 27, 27, 8)         40        
_________________________________________________________________
re_lu (ReLU)                 (None, 27, 27, 8)         0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 9, 9, 8)           0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 8, 8, 8)           264       
_________________________________________________________________
re_lu_1 (ReLU)               (None, 8, 8, 8)           0

# Training

In [23]:
scheduler = LearningRateScheduler(lambda x: 5e-5 if x<80 else 5e-6)

In [24]:
model.fit(X_train,y_train, epochs=100, batch_size=32, validation_data=(X_test,y_test), callbacks=[scheduler])

Train on 54000 samples, validate on 6000 samples
Epoch 1/100
54000/54000 [==============================] - 9s 173us/sample - loss: 2.2655 - acc: 0.1264 - val_loss: 2.2194 - val_acc: 0.2097
Epoch 2/100
54000/54000 [==============================] - 9s 176us/sample - loss: 2.1299 - acc: 0.2839 - val_loss: 2.0118 - val_acc: 0.3477
Epoch 3/100
54000/54000 [==============================] - 10s 184us/sample - loss: 1.8376 - acc: 0.4206 - val_loss: 1.6837 - val_acc: 0.4693
Epoch 4/100
54000/54000 [==============================] - 10s 188us/sample - loss: 1.5641 - acc: 0.5001 - val_loss: 1.4742 - val_acc: 0.5238
Epoch 5/100
54000/54000 [==============================] - 9s 174us/sample - loss: 1.3954 - acc: 0.5496 - val_loss: 1.3373 - val_acc: 0.5657
Epoch 6/100
54000/54000 [==============================] - 9s 173us/sample - loss: 1.2783 - acc: 0.5846 - val_loss: 1.2354 - val_acc: 0.5990
Epoch 7/100
54000/54000 [==============================] - 10s 185us/sample - loss: 1.1906 - acc: 0.613

54000/54000 [==============================] - 9s 162us/sample - loss: 0.5147 - acc: 0.8410 - val_loss: 0.5083 - val_acc: 0.8447
Epoch 59/100
54000/54000 [==============================] - 9s 159us/sample - loss: 0.5108 - acc: 0.8428 - val_loss: 0.5023 - val_acc: 0.8470
Epoch 60/100
54000/54000 [==============================] - 11s 213us/sample - loss: 0.5061 - acc: 0.8449 - val_loss: 0.4989 - val_acc: 0.8473
Epoch 61/100
54000/54000 [==============================] - 12s 213us/sample - loss: 0.5014 - acc: 0.8463 - val_loss: 0.4950 - val_acc: 0.8508
Epoch 62/100
54000/54000 [==============================] - 15s 286us/sample - loss: 0.4956 - acc: 0.8476 - val_loss: 0.4919 - val_acc: 0.8522
Epoch 63/100
54000/54000 [==============================] - 12s 215us/sample - loss: 0.4927 - acc: 0.8493 - val_loss: 0.4903 - val_acc: 0.8533
Epoch 64/100
54000/54000 [==============================] - 14s 260us/sample - loss: 0.4874 - acc: 0.8499 - val_loss: 0.4804 - val_acc: 0.8525
Epoch 65/100
5

## Original acc

In [9]:
preds = model.predict(X_test)

In [10]:
res = np.argmax(preds,axis=1) == np.argmax(y_test,axis=1)
np.sum(res)/len(res)

0.7856666666666666

# Save

In [11]:
model.save("model.h5")

# Convert to TF format

In [12]:
def freeze_session(session, keep_var_names=None, output_names=None, clear_devices=True):
    """
    Freezes the state of a session into a pruned computation graph.

    Creates a new computation graph where variable nodes are replaced by
    constants taking their current value in the session. The new graph will be
    pruned so subgraphs that are not necessary to compute the requested
    outputs are removed.
    @param session The TensorFlow session to be frozen.
    @param keep_var_names A list of variable names that should not be frozen,
                          or None to freeze all the variables in the graph.
    @param output_names Names of the relevant graph outputs.
    @param clear_devices Remove the device directives from the graph for better portability.
    @return The frozen graph definition.
    """
    graph = session.graph
    with graph.as_default():
        freeze_var_names = list(set(v.op.name for v in tf.global_variables()).difference(keep_var_names or []))
        output_names = output_names or []
        output_names += [v.op.name for v in tf.global_variables()]
        input_graph_def = graph.as_graph_def()
        if clear_devices:
            for node in input_graph_def.node:
                node.device = ""
        frozen_graph = tf.graph_util.convert_variables_to_constants(
            session, input_graph_def, output_names, freeze_var_names)
        return frozen_graph

In [13]:
k_inputs = [out.op.name for out in model.inputs]
k_outputs = [out.op.name for out in model.outputs]
k_inputs, k_outputs

(['reshape_input'], ['dense/Sigmoid'])

In [16]:
frozen_graph = freeze_session(K.get_session(), output_names=k_outputs)

Instructions for updating:
Use `tf.compat.v1.graph_util.convert_variables_to_constants`
Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`
INFO:tensorflow:Froze 23 variables.
INFO:tensorflow:Converted 23 variables to const ops.


In [17]:
tf.train.write_graph(frozen_graph, "tf_model", "my_model.pb", as_text=False)

'tf_model/my_model.pb'

# TODO: To TRT

In [18]:
graph = tf.Graph()
sess = tf.InteractiveSession(graph = graph)

with tf.gfile.GFile("./tf_model/my_model.pb", 'rb') as f:
    graph_def = tf.GraphDef()
    graph_def.ParseFromString(f.read())

In [19]:
converter = trt.TrtGraphConverter(
            input_graph_def=graph_def,
            nodes_blacklist=k_outputs,
            max_workspace_size_bytes=1 << 32,
            precision_mode='INT8',
            minimum_segment_size=2,
            is_dynamic_op=True,
            maximum_cached_engines=100)
trt_frozen_graph = converter.convert()

INFO:tensorflow:Linked TensorRT version: (0, 0, 0)
INFO:tensorflow:Loaded TensorRT version: (0, 0, 0)
INFO:tensorflow:Running against TensorRT version 0.0.0


In [20]:
type(trt_frozen_graph)

tensorflow.core.framework.graph_pb2.GraphDef